In [83]:
import polars as pl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import polars as pl
import scipy.stats as stats
import scipy.sparse as sps
import gc
import tqdm

In [84]:
from os import system, getpid, walk
from psutil import Process
from colorama import Fore, Style, init
from IPython.display import display, HTML

def PrintColor(text:str, color = Fore.BLUE, style = Style.BRIGHT):
    print(style + color + text + Style.RESET_ALL)
    
def GetMemUsage():   
    pid = getpid()
    py = Process(pid)
    memory_use = py.memory_info()[0] / 2. ** 30
    return f"RAM memory GB usage = {memory_use :.4}"

PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)


RAM memory GB usage = 25.16


In [85]:
def reduce_polars_df_memory_size(df, set_categorical=True):

    start_mem = df.estimated_size('mb')
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type in [pl.Int16, pl.Int32, pl.Int64]:
            c_min = df[col].fill_null(0).min()
            c_max = df[col].fill_null(0).max()
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df = df.with_columns(pl.col(col).cast(pl.Int8))
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df = df.with_columns(pl.col(col).cast(pl.Int16))
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df = df.with_columns(pl.col(col).cast(pl.Int32))
        elif col_type in [pl.UInt16, pl.UInt32, pl.UInt64]:
            c_min = df[col].fill_null(0).min()
            c_max = df[col].fill_null(0).max()
            if c_min > np.iinfo(np.uint8).min and c_max < np.iinfo(np.uint8).max:
                df = df.with_columns(pl.col(col).cast(pl.UInt8))
            elif c_min > np.iinfo(np.uint16).min and c_max < np.iinfo(np.uint16).max:
                df = df.with_columns(pl.col(col).cast(pl.UInt16))
            elif c_min > np.iinfo(np.uint32).min and c_max < np.iinfo(np.uint32).max:
                df = df.with_columns(pl.col(col).cast(pl.UInt32))
        elif col_type == pl.Float64:
            c_min = df[col].fill_null(0).min()
            c_max = df[col].fill_null(0).max()
            if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                df = df.with_columns(pl.col(col).cast(pl.Float32))

    gc.collect()
    end_mem = df.estimated_size('mb')
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [86]:
articles = pl.read_parquet('/home/ubuntu/dataset/ebnerd_small/articles.parquet')

behaviors_train = pl.read_parquet('/home/ubuntu/dataset/ebnerd_small/train/behaviors.parquet')
history_train = pl.read_parquet('/home/ubuntu/dataset/ebnerd_small/train/history.parquet')

behaviors_val = pl.read_parquet('/home/ubuntu/dataset/ebnerd_small/validation/behaviors.parquet')
history_val = pl.read_parquet('/home/ubuntu/dataset/ebnerd_small/validation/history.parquet')

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)


RAM memory GB usage = 25.27


In [87]:
unique_entities = articles.select('entity_groups').explode('entity_groups')['entity_groups'].unique().to_list()
unique_entities = [e for e in unique_entities if e is not None]
unique_entities

['PER', 'PROD', 'ORG', 'EVENT', 'LOC', 'MISC']

In [88]:
train_ds = pl.read_parquet('/home/ubuntu/small_ds/train/train_ds.parquet')
train_ds

impression_id,user_id,article,target,device_type,read_time,scroll_percentage,is_sso_user,gender,age,is_subscriber,postcode,trendiness_score_1d,trendiness_score_3d,trendiness_score_5d,weekday,hour,trendiness_score_1d/3d,trendiness_score_1d/5d,normalized_trendiness_score_overall,premium,category,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews,total_inviews,total_read_time,total_pageviews/inviews,article_type,article_delay_days,article_delay_hours,Entity_EVENT_Present,…,entropy_impression_endorsement_10h,entropy_impression_total_pageviews/inviews,entropy_impression_mean_JS,entropy_impression_mean_topic_model_cosine,entropy_impression_topics_cosine,entropy_impression_article_delay_hours,entropy_impression_total_pageviews,entropy_impression_total_inviews,entropy_impression_trendiness_score_category,entropy_impression_std_JS,entropy_impression_trendiness_score_5d,entropy_impression_total_read_time,trendiness_score_3d_minus_median_impression,endorsement_10h_minus_median_impression,total_pageviews/inviews_minus_median_impression,mean_JS_minus_median_impression,mean_topic_model_cosine_minus_median_impression,topics_cosine_minus_median_impression,article_delay_hours_minus_median_impression,total_pageviews_minus_median_impression,total_inviews_minus_median_impression,trendiness_score_category_minus_median_impression,std_JS_minus_median_impression,trendiness_score_5d_minus_median_impression,total_read_time_minus_median_impression,mean_JS_l_inf_user,std_JS_l_inf_user,mean_topic_model_cosine_l_inf_user,topics_cosine_l_inf_user,article_delay_hours_l_inf_article,mean_JS_l_inf_article,std_JS_l_inf_article,mean_topic_model_cosine_l_inf_article,topics_cosine_l_inf_article,category_diversity_impression,sentiment_label_diversity_impression,article_type_diversity_impression
u32,u32,i32,i8,i8,f32,f32,bool,i8,i8,bool,i8,i16,i16,i16,i8,i8,f32,f32,f32,bool,i16,f32,str,u32,u8,u8,u16,u32,i32,i32,f32,f32,str,i16,i32,bool,…,f64,f32,f32,f32,f32,f64,f64,f64,f64,f32,f64,f32,f64,f64,f32,f32,f32,f32,f64,f64,f64,f64,f32,f64,f32,f32,f32,f32,f32,f64,f32,f32,f32,f32,u32,u32,u32
149474,139836,9778728,0,2,13.0,null,false,2,null,false,5,150,521,836,3,7,0.287908,0.179426,0.880068,false,142,0.9654,"""Negative""",1,5,18,251,7,22415,220247,1.004828e6,0.101772,"""article_default""",0,0,true,…,1.622597,1.726481,1.747889,1.788942,1.689578,NaN,1.613433,1.737743,1.612282,1.739272,1.676222,1.558951,281.5,-873.5,-0.111306,-0.005177,0.091665,-0.02667,-1.0,-62463.5,-165503.0,4.0,-0.033664,447.5,-4.981345e6,0.447581,0.403328,0.96685,0.36119,0.0,0.233531,0.335164,0.908604,0.301919,4,3,1
149474,139836,9778669,0,2,13.0,null,false,2,null,false,5,85,199,313,3,7,0.427136,0.271565,0.336149,false,118,0.9481,"""Negative""",1,5,11,150,4,74491,373488,4.365609e6,0.199447,"""article_default""",0,1,false,…,1.622597,1.726481,1.747889,1.788942,1.689578,NaN,1.613433,1.737743,1.612282,1.739272,1.676222,1.558951,-40.5,179.5,-0.013631,0.003967,-0.050146,0.015762,0.0,-10387.5,-12262.0,0.0,0.028015,-75.5,-1.620564e6,0.501163,0.632488,0.797317,0.437299,0.043478,0.266627,0.511387,0.789201,0.319083,4,3,1
149474,139836,9778657,1,2,13.0,null,false,2,null,false,5,45,117,183,3,7,0.384615,0.245902,0.197635,false,118,0.8347,"""Neutral""",2,6,31,336,3,108389,478098,7.606737e6,0.226709,"""article_default""",0,1,false,…,1.622597,1.726481,1.747889,1.788942,1.689578,NaN,1.613433,1.737743,1.612282,1.739272,1.676222,1.558951,-122.5,368.5,0.013631,0.05309,0.005468,0.218611,0.0,23510.5,92348.0,0.0,0.051035,-205.5,1.620564e6,0.78901,0.718014,0.863803,0.801144,0.066667,0.350947,0.730242,0.840012,0.604487,4,3,1
150528,143471,9778682,0,2,25.0,null,false,2,null,false,5,69,206,334,3,7,0.334951,0.206587,0.347973,false,498,0.9546,"""Negative""",1,5,20,267,3,143520,455723,9.298546e6,0.314928,"""article_default""",0,1,true,…,1.712983,2.137333,2.093172,2.19456,2.100994,NaN,2.063931,2.158165,1.924178,2.126514,1.946778,2.008479,7.0,204.0,0.088219,0.013219,0.095761,-0.001332,1.0,48

In [89]:
recsys_features = pl.read_parquet('/mnt/ebs_volume_2/recsys2024/features/recsys/small/train/recsys_scores_features.parquet')
recsys_features

impression_id,article,user_id,PureSVDRecommender,ItemKNNCFRecommender,RP3betaRecommender,SLIM_BPR_Cython,PureSVDItemRecommender
u32,i32,u32,f32,f32,f32,f32,f32
556008624,9193263,10068,0.000002,0.0,0.0,0.0,0.00011
556008624,9695098,10068,0.008315,0.002663,0.003995,0.0,0.009694
556008624,9758734,10068,0.000918,0.0,0.006811,0.0,0.003627
385198819,9762352,10068,0.029058,0.005326,0.006742,0.0,0.028035
556008624,9772548,10068,0.0,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…
567760513,9779705,2590637,0.0,0.0,0.0,0.0,0.0
567760515,9779705,2590637,0.0,0.0,0.0,0.0,0.0
567760513,9779723,2590637,0.0,0.0,0.0,0.0,0.0


In [90]:
train_ds = train_ds.join(recsys_features, on=['impression_id', 'article','user_id'], how= 'left')
train_ds

impression_id,user_id,article,target,device_type,read_time,scroll_percentage,is_sso_user,gender,age,is_subscriber,postcode,trendiness_score_1d,trendiness_score_3d,trendiness_score_5d,weekday,hour,trendiness_score_1d/3d,trendiness_score_1d/5d,normalized_trendiness_score_overall,premium,category,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews,total_inviews,total_read_time,total_pageviews/inviews,article_type,article_delay_days,article_delay_hours,Entity_EVENT_Present,…,entropy_impression_article_delay_hours,entropy_impression_total_pageviews,entropy_impression_total_inviews,entropy_impression_trendiness_score_category,entropy_impression_std_JS,entropy_impression_trendiness_score_5d,entropy_impression_total_read_time,trendiness_score_3d_minus_median_impression,endorsement_10h_minus_median_impression,total_pageviews/inviews_minus_median_impression,mean_JS_minus_median_impression,mean_topic_model_cosine_minus_median_impression,topics_cosine_minus_median_impression,article_delay_hours_minus_median_impression,total_pageviews_minus_median_impression,total_inviews_minus_median_impression,trendiness_score_category_minus_median_impression,std_JS_minus_median_impression,trendiness_score_5d_minus_median_impression,total_read_time_minus_median_impression,mean_JS_l_inf_user,std_JS_l_inf_user,mean_topic_model_cosine_l_inf_user,topics_cosine_l_inf_user,article_delay_hours_l_inf_article,mean_JS_l_inf_article,std_JS_l_inf_article,mean_topic_model_cosine_l_inf_article,topics_cosine_l_inf_article,category_diversity_impression,sentiment_label_diversity_impression,article_type_diversity_impression,PureSVDRecommender,ItemKNNCFRecommender,RP3betaRecommender,SLIM_BPR_Cython,PureSVDItemRecommender
u32,u32,i32,i8,i8,f32,f32,bool,i8,i8,bool,i8,i16,i16,i16,i8,i8,f32,f32,f32,bool,i16,f32,str,u32,u8,u8,u16,u32,i32,i32,f32,f32,str,i16,i32,bool,…,f64,f64,f64,f64,f32,f64,f32,f64,f64,f32,f32,f32,f32,f64,f64,f64,f64,f32,f64,f32,f32,f32,f32,f32,f64,f32,f32,f32,f32,u32,u32,u32,f32,f32,f32,f32,f32
149474,139836,9778728,0,2,13.0,null,false,2,null,false,5,150,521,836,3,7,0.287908,0.179426,0.880068,false,142,0.9654,"""Negative""",1,5,18,251,7,22415,220247,1.004828e6,0.101772,"""article_default""",0,0,true,…,NaN,1.613433,1.737743,1.612282,1.739272,1.676222,1.558951,281.5,-873.5,-0.111306,-0.005177,0.091665,-0.02667,-1.0,-62463.5,-165503.0,4.0,-0.033664,447.5,-4.981345e6,0.447581,0.403328,0.96685,0.36119,0.0,0.233531,0.335164,0.908604,0.301919,4,3,1,0.0,0.0,0.0,0.0,0.0
149474,139836,9778669,0,2,13.0,null,false,2,null,false,5,85,199,313,3,7,0.427136,0.271565,0.336149,false,118,0.9481,"""Negative""",1,5,11,150,4,74491,373488,4.365609e6,0.199447,"""article_default""",0,1,false,…,NaN,1.613433,1.737743,1.612282,1.739272,1.676222,1.558951,-40.5,179.5,-0.013631,0.003967,-0.050146,0.015762,0.0,-10387.5,-12262.0,0.0,0.028015,-75.5,-1.620564e6,0.501163,0.632488,0.797317,0.437299,0.043478,0.266627,0.511387,0.789201,0.319083,4,3,1,0.0,0.0,0.0,0.0,0.0
149474,139836,9778657,1,2,13.0,null,false,2,null,false,5,45,117,183,3,7,0.384615,0.245902,0.197635,false,118,0.8347,"""Neutral""",2,6,31,336,3,108389,478098,7.606737e6,0.226709,"""article_default""",0,1,false,…,NaN,1.613433,1.737743,1.612282,1.739272,1.676222,1.558951,-122.5,368.5,0.013631,0.05309,0.005468,0.218611,0.0,23510.5,92348.0,0.0,0.051035,-205.5,1.620564e6,0.78901,0.718014,0.863803,0.801144,0.066667,0.350947,0.730242,0.840012,0.604487,4,3,1,0.0,0.0,0.0,0.0,0.0
150528,143471,9778682,0,2,25.0,null,false,2,null,false,5,69,206,334,3,7,0.334951,0.206587,0.347973,false,498,0.9546,"""Negative""",1,5,20,267,3,143520,455723,9.298546e6,0.314928,"""article_default""",0,1,true,…,NaN,2.063931,2.158165,1.924178,2.126514,1.946778,2.008479,7.0,204.0,0.088219,0.013219,0.095761,-0.001332,1.0,48254.0,82235.0,-47.0,0.013249,21.0,4.223417e6,0.793928,0.732491,0.94067,0.581838,0.041667,0.299855,0.536061,0.891664,0.447169,6,3,1,0.0,0.0,0.0,0.0,0.0
150528,143471,9778669,0,2,25.0,null,false,2,null,fa

In [91]:

NORMALIZE_OVER_IMPRESSION_ID = [
    'PureSVDRecommender','ItemKNNCFRecommender','RP3betaRecommender','SLIM_BPR_Cython','PureSVDItemRecommender'
]
NORMALIZE_OVER_USER_ID = [
    'PureSVDRecommender','ItemKNNCFRecommender','RP3betaRecommender','SLIM_BPR_Cython','PureSVDItemRecommender'

]
NORMALIZE_OVER_ARTICLE = [
    'PureSVDRecommender','ItemKNNCFRecommender','RP3betaRecommender','SLIM_BPR_Cython','PureSVDItemRecommender'

]
train_ds = train_ds.with_columns(
    *[(pl.col(c) / pl.col(c).max().over(pl.col('impression_id'))).alias(f'{c}_l_inf_impression')
      for c in NORMALIZE_OVER_IMPRESSION_ID],
    *[pl.col(c).std().over(pl.col('impression_id')).alias(f'std_impression_{c}')
      for c in NORMALIZE_OVER_IMPRESSION_ID],
    *[pl.col(c).skew().over(pl.col('impression_id')).alias(f'skew_impression_{c}')
      for c in NORMALIZE_OVER_IMPRESSION_ID],
    *[pl.col(c).kurtosis().over(pl.col('impression_id')).alias(f'kurtosis_impression_{c}')
      for c in NORMALIZE_OVER_IMPRESSION_ID],
    *[pl.col(c).entropy().over(pl.col('impression_id')).alias(f'entropy_impression_{c}')
      for c in NORMALIZE_OVER_IMPRESSION_ID],
    *[(pl.col(c) - pl.col(c).median().over(pl.col('impression_id'))).alias(f'{c}_minus_median_impression')
      for c in NORMALIZE_OVER_IMPRESSION_ID],
    *[(pl.col(c) / pl.col(c).max().over(pl.col('user_id'))).alias(f'{c}_l_inf_user')
      for c in NORMALIZE_OVER_USER_ID],
    *[(pl.col(c) / pl.col(c).max().over(pl.col('article'))).alias(f'{c}_l_inf_article')
      for c in NORMALIZE_OVER_ARTICLE],
)


In [92]:
train_ds

impression_id,user_id,article,target,device_type,read_time,scroll_percentage,is_sso_user,gender,age,is_subscriber,postcode,trendiness_score_1d,trendiness_score_3d,trendiness_score_5d,weekday,hour,trendiness_score_1d/3d,trendiness_score_1d/5d,normalized_trendiness_score_overall,premium,category,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews,total_inviews,total_read_time,total_pageviews/inviews,article_type,article_delay_days,article_delay_hours,Entity_EVENT_Present,…,SLIM_BPR_Cython_l_inf_impression,PureSVDItemRecommender_l_inf_impression,std_impression_PureSVDRecommender,std_impression_ItemKNNCFRecommender,std_impression_RP3betaRecommender,std_impression_SLIM_BPR_Cython,std_impression_PureSVDItemRecommender,skew_impression_PureSVDRecommender,skew_impression_ItemKNNCFRecommender,skew_impression_RP3betaRecommender,skew_impression_SLIM_BPR_Cython,skew_impression_PureSVDItemRecommender,kurtosis_impression_PureSVDRecommender,kurtosis_impression_ItemKNNCFRecommender,kurtosis_impression_RP3betaRecommender,kurtosis_impression_SLIM_BPR_Cython,kurtosis_impression_PureSVDItemRecommender,entropy_impression_PureSVDRecommender,entropy_impression_ItemKNNCFRecommender,entropy_impression_RP3betaRecommender,entropy_impression_SLIM_BPR_Cython,entropy_impression_PureSVDItemRecommender,PureSVDRecommender_minus_median_impression,ItemKNNCFRecommender_minus_median_impression,RP3betaRecommender_minus_median_impression,SLIM_BPR_Cython_minus_median_impression,PureSVDItemRecommender_minus_median_impression,PureSVDRecommender_l_inf_user,ItemKNNCFRecommender_l_inf_user,RP3betaRecommender_l_inf_user,SLIM_BPR_Cython_l_inf_user,PureSVDItemRecommender_l_inf_user,PureSVDRecommender_l_inf_article,ItemKNNCFRecommender_l_inf_article,RP3betaRecommender_l_inf_article,SLIM_BPR_Cython_l_inf_article,PureSVDItemRecommender_l_inf_article
u32,u32,i32,i8,i8,f32,f32,bool,i8,i8,bool,i8,i16,i16,i16,i8,i8,f32,f32,f32,bool,i16,f32,str,u32,u8,u8,u16,u32,i32,i32,f32,f32,str,i16,i32,bool,…,f32,f32,f32,f32,f32,f32,f32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
149474,139836,9778728,0,2,13.0,null,false,2,null,false,5,150,521,836,3,7,0.287908,0.179426,0.880068,false,142,0.9654,"""Negative""",1,5,18,251,7,22415,220247,1.004828e6,0.101772,"""article_default""",0,0,true,…,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
149474,139836,9778669,0,2,13.0,null,false,2,null,false,5,85,199,313,3,7,0.427136,0.271565,0.336149,false,118,0.9481,"""Negative""",1,5,11,150,4,74491,373488,4.365609e6,0.199447,"""article_default""",0,1,false,…,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
149474,139836,9778657,1,2,13.0,null,false,2,null,false,5,45,117,183,3,7,0.384615,0.245902,0.197635,false,118,0.8347,"""Neutral""",2,6,31,336,3,108389,478098,7.606737e6,0.226709,"""article_default""",0,1,false,…,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
150528,143471,9778682,0,2,25.0,null,false,2,null,false,5,69,206,334,3,7,0.334951,0.206587,0.347973,false,498,0.9546,"""Negative""",1,5,20,267,3,143520,455723,9.298546e6,0.314928,"""article_default""",0,1,true,…,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
150528,143471,9778669,0,2,25.0,null,false,2,null,false,5,85,199,313,3,7,0.427136,0.271565,0.336149,false,118,0.9481,"""Negative""",1,5,11,150,4,74491,373488,4.365609e6,0.199447,"""article_default""",0,1,false,…,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,

In [93]:
from catboost import CatBoostClassifier
from catboost import CatBoostRanker

## Classifier


In [94]:
"""train_ds = train_ds.drop(['impression_id', 'article', 'user_id']).to_pandas()

categorical_columns = ['device_type', 'is_sso_user', 'gender', 'is_subscriber', 'weekday',
                       'premium', 'category', 'sentiment_label', 'is_new_article', 'is_already_seen_article',
                       'MostFrequentCategory', 'MostFrequentWeekday', 'IsFavouriteCategory',
                       'article_type', 'postcode']
categorical_columns += [f'Entity_{entity}_Present' for entity in unique_entities]
train_ds[categorical_columns] = train_ds[categorical_columns].astype('category')

X = train_ds.drop(columns=['target'])
y = train_ds['target']
"""


"train_ds = train_ds.drop(['impression_id', 'article', 'user_id']).to_pandas()\n\ncategorical_columns = ['device_type', 'is_sso_user', 'gender', 'is_subscriber', 'weekday',\n                       'premium', 'category', 'sentiment_label', 'is_new_article', 'is_already_seen_article',\n                       'MostFrequentCategory', 'MostFrequentWeekday', 'IsFavouriteCategory',\n                       'article_type', 'postcode']\ncategorical_columns += [f'Entity_{entity}_Present' for entity in unique_entities]\ntrain_ds[categorical_columns] = train_ds[categorical_columns].astype('category')\n\nX = train_ds.drop(columns=['target'])\ny = train_ds['target']\n"

In [95]:
"""
# just a simple model to be able to run an evaluation
model = CatBoostClassifier(cat_features=categorical_columns, iterations=1000, rsm=0.7, subsample=0.5)

model.fit(X, y, verbose=25)

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)
"""

'\n# just a simple model to be able to run an evaluation\nmodel = CatBoostClassifier(cat_features=categorical_columns, iterations=1000, rsm=0.7, subsample=0.5)\n\nmodel.fit(X, y, verbose=25)\n\ngc.collect()\nPrintColor(f"\n" + GetMemUsage(), color = Fore.RED)\n'

## Ranker


In [96]:
train_ds = train_ds.drop(['article', 'user_id']).to_pandas().sort_values(by='impression_id')
groups = train_ds['impression_id'].copy()

categorical_columns = ['device_type', 'is_sso_user', 'gender', 'is_subscriber', 'weekday',
                       'premium', 'category', 'sentiment_label', 'is_new_article', 'is_already_seen_article',
                       'MostFrequentCategory', 'MostFrequentWeekday', 'IsFavouriteCategory',
                       'article_type', 'postcode']
categorical_columns += [f'Entity_{entity}_Present' for entity in unique_entities]
train_ds[categorical_columns] = train_ds[categorical_columns].astype('category')

X = train_ds.drop(columns=['target', 'impression_id'])
y = train_ds['target']

In [97]:

model = CatBoostRanker(cat_features=categorical_columns, iterations=2000, depth=8, colsample_bylevel=0.5)

model.fit(X, y, group_id=groups, verbose=25)

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)


Groupwise loss function. OneHotMaxSize set to 10
0:	total: 683ms	remaining: 22m 44s
25:	total: 16.3s	remaining: 20m 37s
50:	total: 31.4s	remaining: 19m 58s
75:	total: 46.8s	remaining: 19m 44s
100:	total: 1m 2s	remaining: 19m 30s
125:	total: 1m 17s	remaining: 19m 16s
150:	total: 1m 32s	remaining: 18m 58s
175:	total: 1m 48s	remaining: 18m 43s
200:	total: 2m 4s	remaining: 18m 33s
225:	total: 2m 20s	remaining: 18m 19s
250:	total: 2m 35s	remaining: 18m 2s
275:	total: 2m 50s	remaining: 17m 46s
300:	total: 3m 6s	remaining: 17m 29s
325:	total: 3m 21s	remaining: 17m 14s
350:	total: 3m 37s	remaining: 17m
375:	total: 3m 52s	remaining: 16m 44s
400:	total: 4m 7s	remaining: 16m 28s
425:	total: 4m 23s	remaining: 16m 12s
450:	total: 4m 38s	remaining: 15m 56s
475:	total: 4m 53s	remaining: 15m 39s
500:	total: 5m 8s	remaining: 15m 24s
525:	total: 5m 24s	remaining: 15m 8s
550:	total: 5m 39s	remaining: 14m 53s
575:	total: 5m 55s	remaining: 14m 38s
600:	total: 6m 11s	remaining: 14m 23s
625:	total: 6m 26s	re

In [98]:
val_ds = pl.read_parquet('/home/ubuntu/small_ds/validation/val_ds.parquet')
val_ds

impression_id,article,user_id,target,device_type,read_time,scroll_percentage,is_sso_user,gender,age,is_subscriber,postcode,trendiness_score_1d,trendiness_score_3d,trendiness_score_5d,weekday,hour,trendiness_score_1d/3d,trendiness_score_1d/5d,normalized_trendiness_score_overall,premium,category,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews,total_inviews,total_read_time,total_pageviews/inviews,article_type,article_delay_days,article_delay_hours,Entity_EVENT_Present,…,entropy_impression_endorsement_10h,entropy_impression_total_pageviews/inviews,entropy_impression_mean_JS,entropy_impression_mean_topic_model_cosine,entropy_impression_topics_cosine,entropy_impression_article_delay_hours,entropy_impression_total_pageviews,entropy_impression_total_inviews,entropy_impression_trendiness_score_category,entropy_impression_std_JS,entropy_impression_trendiness_score_5d,entropy_impression_total_read_time,trendiness_score_3d_minus_median_impression,endorsement_10h_minus_median_impression,total_pageviews/inviews_minus_median_impression,mean_JS_minus_median_impression,mean_topic_model_cosine_minus_median_impression,topics_cosine_minus_median_impression,article_delay_hours_minus_median_impression,total_pageviews_minus_median_impression,total_inviews_minus_median_impression,trendiness_score_category_minus_median_impression,std_JS_minus_median_impression,trendiness_score_5d_minus_median_impression,total_read_time_minus_median_impression,mean_JS_l_inf_user,std_JS_l_inf_user,mean_topic_model_cosine_l_inf_user,topics_cosine_l_inf_user,article_delay_hours_l_inf_article,mean_JS_l_inf_article,std_JS_l_inf_article,mean_topic_model_cosine_l_inf_article,topics_cosine_l_inf_article,category_diversity_impression,sentiment_label_diversity_impression,article_type_diversity_impression
u32,i32,u32,i8,i8,f32,f32,bool,i8,i8,bool,i8,i16,i16,i16,i8,i8,f32,f32,f32,bool,i16,f32,str,u32,u8,u8,u16,u32,i32,i32,f32,f32,str,i16,i32,bool,…,f64,f32,f32,f32,f32,f64,f64,f64,f64,f32,f64,f32,f64,f64,f32,f32,f32,f32,f64,f64,f64,f64,f32,f64,f32,f32,f32,f32,f32,f64,f32,f32,f32,f32,u32,u32,u32
205373625,9785030,1801226,0,2,251.0,null,true,0,null,true,5,30,80,162,7,17,0.375,0.185185,0.146789,false,414,0.9167,"""Positive""",3,6,31,426,3,37391,349453,2.850225e6,0.106999,"""article_default""",0,2,false,…,3.613138,3.589502,3.595951,3.758065,3.631038,NaN,3.447933,3.399587,3.603802,3.672916,3.567628,3.488753,-139.0,69.0,-0.021321,-0.059766,0.077358,-0.207508,-3.0,-15276.5,-34312.5,-67.0,-0.035288,-253.0,-770304.0,0.291109,0.442209,0.972551,0.25217,0.025974,0.253,0.421015,0.879186,0.272875,8,3,3
456698625,9553264,1757180,0,2,34.0,null,false,2,null,false,5,55,105,190,3,5,0.52381,0.289474,0.173267,true,457,0.8349,"""Neutral""",1,6,20,718,3,null,null,null,null,"""article_default""",162,3908,false,…,2.414501,2.598626,2.587271,2.703429,2.621012,NaN,2.487291,2.576228,2.518426,2.683312,2.546191,2.466265,-80.0,-377.0,null,-0.007509,-0.09586,-0.077113,3906.0,null,null,-96.0,0.000809,-89.0,null,0.324542,0.586527,0.720961,0.291917,0.993644,0.10526,0.269075,0.808239,0.179034,5,3,1
46558736,7213923,1885995,0,2,25.0,null,false,2,null,false,5,26,64,94,1,6,0.40625,0.276596,0.105263,true,565,0.8109,"""Neutral""",5,10,42,1048,4,null,null,null,null,"""article_default""",1766,42385,false,…,2.5383,2.609011,2.512937,2.704911,2.545427,NaN,2.441637,2.554746,2.483112,2.606141,2.522325,2.387892,-98.0,-1373.0,null,-0.074603,0.002056,-0.228412,42383.0,null,null,-95.0,-0.09143,-217.0,null,0.093726,0.196071,0.89184,0.124086,0.998281,0.045365,0.194879,0.848475,0.144924,6,3,1
470321755,9504105,594528,0,2,0.0,100.0,false,2,null,false,5,84,249,393,2,19,0.337349,0.21374,0.394612,true,142,0.8874,"""Positive""",5,7,44,836,2,null,null,null,null,"""article_default""",203,4895,true,…,3.224331,3.149038,NaN,3.427022,NaN,NaN,3.018402,2.828171,3.191193,NaN,3.153275,3.059159,121.0,-737.0,null,0.20034,0.079726,0.402638,4892.0,null,null,27.0,0.075762,185.0,null,0.495476,0.434085,0.

In [99]:
recsys_features = pl.read_parquet('/mnt/ebs_volume_2/recsys2024/features/recsys/small/validation/recsys_scores_features.parquet')
recsys_features

impression_id,article,user_id,PureSVDRecommender,ItemKNNCFRecommender,RP3betaRecommender,SLIM_BPR_Cython,PureSVDItemRecommender
u32,i32,u32,f32,f32,f32,f32,f32
80366312,7213923,10200,3.1867e-8,0.0,0.0,0.0,0.0
225664684,7594265,10200,1.0059e-8,0.0,0.0,0.0,0.0
225664679,7594265,10200,1.0059e-8,0.0,0.0,0.0,0.0
225664678,7594265,10200,1.0059e-8,0.0,0.0,0.0,0.0
322455613,9142564,10200,-1.0755e-8,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…
185972553,9790278,2590637,0.0,0.0,0.0,0.0,0.0
185972553,9790406,2590637,0.0,0.0,0.0,0.0,0.0
185972553,9790414,2590637,0.0,0.0,0.0,0.0,0.0


In [100]:
val_ds = val_ds.join(recsys_features, on=['article','impression_id','user_id'], how='left')
val_ds

impression_id,article,user_id,target,device_type,read_time,scroll_percentage,is_sso_user,gender,age,is_subscriber,postcode,trendiness_score_1d,trendiness_score_3d,trendiness_score_5d,weekday,hour,trendiness_score_1d/3d,trendiness_score_1d/5d,normalized_trendiness_score_overall,premium,category,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,total_pageviews,total_inviews,total_read_time,total_pageviews/inviews,article_type,article_delay_days,article_delay_hours,Entity_EVENT_Present,…,entropy_impression_article_delay_hours,entropy_impression_total_pageviews,entropy_impression_total_inviews,entropy_impression_trendiness_score_category,entropy_impression_std_JS,entropy_impression_trendiness_score_5d,entropy_impression_total_read_time,trendiness_score_3d_minus_median_impression,endorsement_10h_minus_median_impression,total_pageviews/inviews_minus_median_impression,mean_JS_minus_median_impression,mean_topic_model_cosine_minus_median_impression,topics_cosine_minus_median_impression,article_delay_hours_minus_median_impression,total_pageviews_minus_median_impression,total_inviews_minus_median_impression,trendiness_score_category_minus_median_impression,std_JS_minus_median_impression,trendiness_score_5d_minus_median_impression,total_read_time_minus_median_impression,mean_JS_l_inf_user,std_JS_l_inf_user,mean_topic_model_cosine_l_inf_user,topics_cosine_l_inf_user,article_delay_hours_l_inf_article,mean_JS_l_inf_article,std_JS_l_inf_article,mean_topic_model_cosine_l_inf_article,topics_cosine_l_inf_article,category_diversity_impression,sentiment_label_diversity_impression,article_type_diversity_impression,PureSVDRecommender,ItemKNNCFRecommender,RP3betaRecommender,SLIM_BPR_Cython,PureSVDItemRecommender
u32,i32,u32,i8,i8,f32,f32,bool,i8,i8,bool,i8,i16,i16,i16,i8,i8,f32,f32,f32,bool,i16,f32,str,u32,u8,u8,u16,u32,i32,i32,f32,f32,str,i16,i32,bool,…,f64,f64,f64,f64,f32,f64,f32,f64,f64,f32,f32,f32,f32,f64,f64,f64,f64,f32,f64,f32,f32,f32,f32,f32,f64,f32,f32,f32,f32,u32,u32,u32,f32,f32,f32,f32,f32
205373625,9785030,1801226,0,2,251.0,null,true,0,null,true,5,30,80,162,7,17,0.375,0.185185,0.146789,false,414,0.9167,"""Positive""",3,6,31,426,3,37391,349453,2.850225e6,0.106999,"""article_default""",0,2,false,…,NaN,3.447933,3.399587,3.603802,3.672916,3.567628,3.488753,-139.0,69.0,-0.021321,-0.059766,0.077358,-0.207508,-3.0,-15276.5,-34312.5,-67.0,-0.035288,-253.0,-770304.0,0.291109,0.442209,0.972551,0.25217,0.025974,0.253,0.421015,0.879186,0.272875,8,3,3,0.0,0.0,0.0,0.0,0.0
456698625,9553264,1757180,0,2,34.0,null,false,2,null,false,5,55,105,190,3,5,0.52381,0.289474,0.173267,true,457,0.8349,"""Neutral""",1,6,20,718,3,null,null,null,null,"""article_default""",162,3908,false,…,NaN,2.487291,2.576228,2.518426,2.683312,2.546191,2.466265,-80.0,-377.0,null,-0.007509,-0.09586,-0.077113,3906.0,null,null,-96.0,0.000809,-89.0,null,0.324542,0.586527,0.720961,0.291917,0.993644,0.10526,0.269075,0.808239,0.179034,5,3,1,0.000609,0.0,0.0,0.0,0.002461
46558736,7213923,1885995,0,2,25.0,null,false,2,null,false,5,26,64,94,1,6,0.40625,0.276596,0.105263,true,565,0.8109,"""Neutral""",5,10,42,1048,4,null,null,null,null,"""article_default""",1766,42385,false,…,NaN,2.441637,2.554746,2.483112,2.606141,2.522325,2.387892,-98.0,-1373.0,null,-0.074603,0.002056,-0.228412,42383.0,null,null,-95.0,-0.09143,-217.0,null,0.093726,0.196071,0.89184,0.124086,0.998281,0.045365,0.194879,0.848475,0.144924,6,3,1,0.003138,0.001332,0.000853,0.0,0.010064
470321755,9504105,594528,0,2,0.0,100.0,false,2,null,false,5,84,249,393,2,19,0.337349,0.21374,0.394612,true,142,0.8874,"""Positive""",5,7,44,836,2,null,null,null,null,"""article_default""",203,4895,true,…,NaN,3.018402,2.828171,3.191193,NaN,3.153275,3.059159,121.0,-737.0,null,0.20034,0.079726,0.402638,4892.0,null,null,27.0,0.075762,185.0,null,0.495476,0.434085,0.963454,0.572476,0.992699,0.629829,0.499302,0.864027,0.71809,8,3,2,0.000079,0.0,0.0,0.0,0.000225
214849994,9279095,811018,0,1,12.0,null,false,2,null,false,5,10,48

In [101]:

NORMALIZE_OVER_IMPRESSION_ID = [
    'PureSVDRecommender','ItemKNNCFRecommender','RP3betaRecommender','SLIM_BPR_Cython','PureSVDItemRecommender'
]

NORMALIZE_OVER_USER_ID = [
    'PureSVDRecommender','ItemKNNCFRecommender','RP3betaRecommender','SLIM_BPR_Cython','PureSVDItemRecommender'

]
NORMALIZE_OVER_ARTICLE = [
    'PureSVDRecommender','ItemKNNCFRecommender','RP3betaRecommender','SLIM_BPR_Cython','PureSVDItemRecommender'

]


val_ds = val_ds.with_columns(
    *[(pl.col(c) / pl.col(c).max().over(pl.col('impression_id'))).alias(f'{c}_l_inf_impression')
      for c in NORMALIZE_OVER_IMPRESSION_ID],
    *[pl.col(c).std().over(pl.col('impression_id')).alias(f'std_impression_{c}')
      for c in NORMALIZE_OVER_IMPRESSION_ID],
    *[pl.col(c).skew().over(pl.col('impression_id')).alias(f'skew_impression_{c}')
      for c in NORMALIZE_OVER_IMPRESSION_ID],
    *[pl.col(c).kurtosis().over(pl.col('impression_id')).alias(f'kurtosis_impression_{c}')
      for c in NORMALIZE_OVER_IMPRESSION_ID],
    *[pl.col(c).entropy().over(pl.col('impression_id')).alias(f'entropy_impression_{c}')
      for c in NORMALIZE_OVER_IMPRESSION_ID],
    *[(pl.col(c) - pl.col(c).median().over(pl.col('impression_id'))).alias(f'{c}_minus_median_impression')
      for c in NORMALIZE_OVER_IMPRESSION_ID],
    *[(pl.col(c) / pl.col(c).max().over(pl.col('user_id'))).alias(f'{c}_l_inf_user')
      for c in NORMALIZE_OVER_USER_ID],
    *[(pl.col(c) / pl.col(c).max().over(pl.col('article'))).alias(f'{c}_l_inf_article')
      for c in NORMALIZE_OVER_ARTICLE],
)


In [108]:
val_ds.shape

(2928942, 300)

In [103]:
"""# impression_id will be later useful for evaluation
val_ds_pandas = val_ds.drop(['impression_id', 'article', 'user_id']).to_pandas()

val_ds_pandas[categorical_columns] = val_ds_pandas[categorical_columns].astype('category')

X_val = val_ds_pandas.drop(columns=['target'])
y_val = val_ds_pandas['target']

val_ds = val_ds.with_columns(pl.Series(model.predict_proba(X_val)[:, 1]).alias('prediction'))
val_ds.select(['impression_id', 'target', 'prediction'])
"""

"# impression_id will be later useful for evaluation\nval_ds_pandas = val_ds.drop(['impression_id', 'article', 'user_id']).to_pandas()\n\nval_ds_pandas[categorical_columns] = val_ds_pandas[categorical_columns].astype('category')\n\nX_val = val_ds_pandas.drop(columns=['target'])\ny_val = val_ds_pandas['target']\n\nval_ds = val_ds.with_columns(pl.Series(model.predict_proba(X_val)[:, 1]).alias('prediction'))\nval_ds.select(['impression_id', 'target', 'prediction'])\n"

In [104]:
# impression_id will be later useful for evaluation
val_ds_pandas = val_ds.drop(['impression_id', 'article', 'user_id']).to_pandas()

val_ds_pandas[categorical_columns] = val_ds_pandas[categorical_columns].astype('category')

X_val = val_ds_pandas.drop(columns=['target'])
y_val = val_ds_pandas['target']

val_ds = val_ds.with_columns(pl.Series(model.predict(X_val)).alias('prediction'))
val_ds.select(['impression_id', 'target', 'prediction'])

impression_id,target,prediction
u32,i8,f64
205373625,0,-1.848472
456698625,0,-1.008827
46558736,0,-3.432989
470321755,0,-2.496034
214849994,0,-3.160967
…,…,…
184283316,0,-1.126955
90104855,0,-1.723605
257661167,0,-1.129269


In [105]:
evaluation_ds = val_ds.group_by('impression_id').agg(pl.col('target'), pl.col('prediction'))
evaluation_ds

impression_id,target,prediction
u32,list[i8],list[f64]
368499364,"[0, 0, … 0]","[0.389976, -0.927275, … -2.983329]"
340400934,"[1, 0, … 0]","[1.343857, -0.040679, … -1.381651]"
68759830,"[0, 0, … 0]","[-1.22962, -0.646782, … 0.289087]"
155145288,"[1, 0, … 0]","[0.075821, -0.478029, … 0.184552]"
120071454,"[0, 0, … 0]","[1.07762, 0.528619, … 1.563752]"
…,…,…
253235257,"[0, 0, … 0]","[0.733294, -1.865555, … 0.179942]"
258876929,"[0, 0, … 0]","[-0.739569, -2.499529, … 2.472374]"
193661407,"[0, 0, … 0]","[-1.551357, 1.532685, … -3.534684]"


In [106]:
import sys
sys.path.append('/home/ubuntu/fastauc')

In [107]:
from fastauc.fast_auc import CppAuc
cpp_auc = CppAuc()
np.mean([cpp_auc.roc_auc_score(np.array(y_t).astype(bool), np.array(y_s).astype(np.float32)) 
                 for y_t, y_s in zip(evaluation_ds['target'].to_list(), 
                                     evaluation_ds['prediction'].to_list())])

0.7918986084369177